# Analysis and visualization of soil fungal communities 

### 1) Load packages

In [ ]:
## install.packages(phyloseq)
## install.packages(ggplot2)

In [ ]:
library(ggplot2)
library(phyloseq)

### 2) Load fungal taxa table 

In [1]:
otu = read.delim("/Users/jessicahoch/Downloads/green_roof_mar18_otu_table_r13000_wTaxa.txt")

### 3) Set taxa IDs as row names 

In [2]:
rownames(otu) = otu$OTU_ID

### 4) Get rid of taxanomic information 

In [3]:
itsMap = as.matrix(otu[,-(1:8)])

### 5) Create dataframe with taxanomic information

In [12]:
taxa = as.matrix(otu[,c(2:8)])

### 6) Load metadata 

In [14]:
meta = read.delim("/Users/jessicahoch/Downloads/fungal-otu-metadata.txt")

### 7) Make a data frame of sample names for merging 

In [16]:
sample = data.frame(colnames(itsMap))

### 8) Transform the OTU table

In [ ]:
X_its = otu_table(itsMap, taxa_are_rows = TRUE)
tax = tax_table(taxa)

### 9) Combine the transformed tables to a phyloseq object 

In [ ]:
physeq = phyloseq(X_its, tax)

### 10) Add metadata to the phyloseq object 

In [ ]:
sampledata = sample_data(data.frame(meta,row.names=sample_names(physeq),stringsAsFactors=FALSE))
Xphyseq = merge_phyloseq(physeq, sampledata)

### 11) Create function that scales the data 

In [ ]:
scale_reads <- function(physeq,n) {
  physeq.scale <-
    transform_sample_counts(physeq, function(x) {
      (n * x/sum(x))
    })
  otu_table(physeq.scale) <- floor(otu_table(physeq.scale))
  physeq.scale <- prune_taxa(taxa_sums(physeq.scale) > 10, physeq.scale)
  return(physeq.scale)
}

In [ ]:
Xphyseq = scale_reads(Xphyseq_jh, n=1E6)


### 12) Create dissimilarity matrix (ordinate function)

In [ ]:
ord = function(physeq_object) {
  ordinate(
    physeq = physeq_object, 
    method = "NMDS", 
    distance = "bray"
  )
}


In [ ]:
Fphyseq = ord(Xphyseq)
stressplot(Fphyseq)


### 13) Visualization (NMDS plot)

In [ ]:
Green_Roof = plot_ordination(
  physeq = Xphyseq,
  ordination = Fphyseq,
  color = ("Veg_Type")) + 
  theme_classic() + 
  ggtitle(element_blank()) +
  geom_point(size = 3)+
  scale_color_manual(labels = c("Mixed-vegetation", "Sedum"), values = c("saddlebrown", "lightseagreen"))+
  theme(legend.title=element_blank())+
  theme(axis.text = element_text(size=16))+
  theme(axis.title=element_text(size=16))+
  theme(legend.text = element_text(size=20))
Green_Roof

### 14) Analysis of similarity 

In [ ]:
green_roof <- phyloseq::distance(Xphyseq, method = "bray")
green_roof_samples <- data.frame(sample_data(Xphyseq))
anosim(green_roof, green_roof_samples$Veg_Type)
### Sedum vs mixed-veg --> p<0.001